This notebook prepare the catalogues that will be analysed by CIGALE for SED fitting and physical parameter estimation.

In [1]:
import numpy as np
import os
os.environ['LOG_LEVEL'] = 'INFO'
import glob
from astropy.table import Table

from herschelhelp.filters import correct_galactic_extinction
from herschelhelp.external import convert_table_for_cigale

In [2]:
SUFFIX = '20180221'
file_list = glob.glob('../../dmu32/dmu32_XMM-LSS/data/tiles/xmm-lss_{}_cigale_*.fits'.format(SUFFIX))

In [3]:
#len(master_catalogue)

# Best sources

Define a good far-IR measurement as:
- an existing flux in the band;
- the flag from XID+ must not be set;
- the signal to noise ratio must be over 2.

In [4]:
# good = {}
# for band in ['pacs_green', 'pacs_red', 'spire_250', 'spire_350', 'spire_500']:
#     good[band] = (~np.isnan(master_catalogue['f_{}'.format(band)]) & 
#                                    ~master_catalogue['flag_{}'.format(band)])
#     good[band][good[band]] &= (master_catalogue[good[band]]['f_{}'.format(band)] /
#                               master_catalogue[good[band]]['ferr_{}'.format(band)] >= 2)

We will keep only sources with at leat 2 good far-IR measurements (we may actually use less sources are not all may have a redshift).

In [5]:
# combined_good = np.sum(list(good.values()), axis=0) >= 2

In [6]:
# print("Number of good sources: {}".format(np.sum(combined_good)))

# Main catalogue for CIGALE

In [7]:
# best_catalogue = master_catalogue[combined_good].copy()

In [8]:
# Correction for galactic extinction
#best_catalogue = correct_galactic_extinction(best_catalogue, inplace=True)

In [9]:
# Convertion to CIGALE format
#best_catalogue = convert_table_for_cigale(best_catalogue, inplace=True, remove_zerofluxes=True)

## Band selection

We want to use only one filter for similar bands. We define an order of preference and set to NaN the flux in the lower prefered bands when a prefered band is available. Some band may have a 0 flux, we set there values to NaN.

In [10]:
u_bands = [                         "megacam_u"]
g_bands = [            "suprime_g", "megacam_g", "decam_g", "gpc1_g" ]
r_bands = [            "suprime_r", "megacam_r", "decam_r", "gpc1_r" ]
i_bands = [            "suprime_i", "megacam_i", "decam_i", "gpc1_i" ]
z_bands = ["vista_z",  "suprime_z", "megacam_z", "decam_z", "gpc1_z" ]
y_bands = ["vista_y",  "suprime_y", "megacam_y", "decam_y", "gpc1_y" ]
j_bands = ["vista_j",  "wircam_j",  "ukidss_j"]
h_bands = ["vista_h",  "wircam_h",  "ukidss_h"]
k_bands = ["vista_ks", "wircam_ks"] #, "ukidss_k"]

def remove_unneeded_fluxes(list_of_bands):
    for band_idx, band in enumerate(list_of_bands[:-1]):
        mask = ~np.isnan(best_catalogue[band])
        for lower_band in list_of_bands[band_idx+1:]:
            best_catalogue[lower_band][mask] = np.nan
            best_catalogue["{}_err".format(lower_band)][mask] = np.nan

In [11]:
# remove_unneeded_fluxes(g_bands)
# remove_unneeded_fluxes(u_bands)
# remove_unneeded_fluxes(r_bands)
# remove_unneeded_fluxes(i_bands)
# remove_unneeded_fluxes(z_bands)
# remove_unneeded_fluxes(y_bands)

In [12]:
#best_catalogue.write("data_tmp/CDFS-SWIRE_cigale_best_extcor_20180129.fits")

# Catalogue using spectroscopic redshift

In [13]:
for tile in file_list:
        
    file_num = tile.split('_')[4].split('.')[0]
    best_catalogue = Table.read(tile)
    optnir = ((best_catalogue['flag_optnir_det'] == 3) 
          | (best_catalogue['flag_optnir_det'] == 7))
    
    best_catalogue = best_catalogue[optnir]


    best_catalogue = best_catalogue[~np.isnan(best_catalogue["redshift"])]

    print("Number of sources with z-spec in file {}: {}".format(file_num, len(best_catalogue)))

    # Correction for galactic extinction
    best_catalogue = correct_galactic_extinction(best_catalogue, inplace=True)

    # Convertion to CIGALE format

    best_catalogue = convert_table_for_cigale(best_catalogue, inplace=True, remove_zerofluxes=True)

    remove_unneeded_fluxes(g_bands)
    remove_unneeded_fluxes(u_bands)
    remove_unneeded_fluxes(r_bands)
    remove_unneeded_fluxes(i_bands)
    remove_unneeded_fluxes(z_bands)
    remove_unneeded_fluxes(y_bands)
    remove_unneeded_fluxes(j_bands)
    remove_unneeded_fluxes(h_bands)
    remove_unneeded_fluxes(k_bands)

    best_catalogue.write("data_tmp/tiles/XMM-LSS_cigale_optnir_extcor_zspec_{}_{}.fits".format(SUFFIX, file_num), overwrite=True)

Number of sources with z-spec in file 40: 1896


ERROR:herschelhelp.filters:The filters are missing in the database: hawki_y.
The are present in these columns: ferr_hawki_y.
INFO:herschelhelp.external:For 7 sources, the band megacam_u should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 4 sources, the band suprime_b should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 2 sources, the band suprime_g should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 2 sources, the band decam_g should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 2 sources, the band megacam_g should not be used because it overlaps or is bel

Number of sources with z-spec in file 2: 1035


ERROR:herschelhelp.filters:The filters are missing in the database: hawki_y.
The are present in these columns: ferr_hawki_y.
INFO:herschelhelp.external:For 4 sources, the band megacam_u should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 3 sources, the band suprime_b should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 2 sources, the band suprime_g should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 2 sources, the band decam_g should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 2 sources, the band megacam_g should not be used because it overlaps or is bel

Number of sources with z-spec in file 71: 1539


...

ERROR:herschelhelp.filters:The filters are missing in the database: hawki_y.
The are present in these columns: ferr_hawki_y.
INFO:herschelhelp.external:For 35 sources, the band megacam_u should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 19 sources, the band suprime_b should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 10 sources, the band suprime_g should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 13 sources, the band decam_g should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 10 sources, the band megacam_g should not be used because it overlaps or i

Number of sources with z-spec in file 56: 1977


ERROR:herschelhelp.filters:The filters are missing in the database: hawki_y.
The are present in these columns: ferr_hawki_y.
INFO:herschelhelp.external:For 9 sources, the band megacam_u should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 5 sources, the band suprime_b should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 3 sources, the band suprime_g should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 3 sources, the band decam_g should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 3 sources, the band megacam_g should not be used because it overlaps or is bel

Number of sources with z-spec in file 34: 1223


ERROR:herschelhelp.filters:The filters are missing in the database: hawki_y.
The are present in these columns: ferr_hawki_y.
INFO:herschelhelp.external:For 6 sources, the band megacam_u should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 2 sources, the band suprime_b should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 1 sources, the band suprime_g should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 1 sources, the band decam_g should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 1 sources, the band megacam_g should not be used because it overlaps or is bel